# Nouveau projet flutter

***Dépendances du script de nouveau projet:***
* *flutter*
* *git*
* *github cli*
* *python 3.10*
* pip: pyyaml

In [ ]:
## Importation ##
import os, re
import platform
import subprocess
import shlex
import yaml
import shutil

## variables globales ##
# inputs
PROJET_NAME = "projet_test"  # lowercase_with_underscores YourTurn
PLATFORMS = ["linux", "windows"]  # "linux", "windows", "macos", "apk", "ios", "web"
DESCRIPTION = (
    "projet test de l’éditeur de nouveau projet Flutter"  # description du projet
)
CHEMIN_RACINE = f"C:\\Users\\user\\Documents\\Programmation\\Applications"
ORGANISATION = "BernardonDev-Data"
USER = "Theobernardon"
if not ORGANISATION:
    ORGANISATION = USER  # si pas d'organisation, on utilise le nom d'utilisateur
GITHUB_PROJECT = f"https://github.com/{ORGANISATION}/{PROJET_NAME}"
DISPONIBILITE = 'private'  # 'public' ou 'private'

## variables de configuration ##
# chemins
CHEMIN_NEW_PROJET = CHEMIN_RACINE + "\\" + PROJET_NAME + "\\"
GITHUB_FOLDER = CHEMIN_NEW_PROJET + ".github\\"
TEMPLATES_FOLDER = GITHUB_FOLDER + "ISSUE_TEMPLATE\\"
WORKFLOW_FOLDER = GITHUB_FOLDER + "workflows\\"
ASSETS_FOLDER = CHEMIN_NEW_PROJET + "assets\\"
IMAGES_FOLDER = ASSETS_FOLDER + "images\\"
INNOSETUP_FILE = CHEMIN_NEW_PROJET + "installer.iss"
PUBSPEC_FILE = CHEMIN_NEW_PROJET + "pubspec.yaml"
IMAGES_RELATIF_FOLDER = IMAGES_FOLDER.replace(CHEMIN_NEW_PROJET, "")

# versions
versions = re.split(
    r"â€¢|\n",
    subprocess.run(["flutter", "--version"], capture_output=True, text=True).stdout,
)


## fonctions ##
def generate_guid():
    system = platform.system()
    if system == "Windows":
        result = subprocess.run(
            ["powershell", "-Command", "[guid]::NewGuid().ToString()"],
            capture_output=True,
            text=True,
            check=True,
        )
    else:
        result = subprocess.run(["uuidgen"], capture_output=True, text=True, check=True)
    return result.stdout.strip()


def process_cmd(
    commandes_lignes: list[str],
    kwargs: dict[str, str] = {},
    return_out: bool = False,
    print_out: bool = True,
):
    def split_for_process_cmd(commande_ligne: str):
        lex = shlex.shlex(commande_ligne, posix=True)
        lex.whitespace_split = True
        lex.escape = ""
        list_lex = []
        for text in list(lex):
            # Si l'argument contient un espace, on le quote à nouveau
            if re.search(r" ", text) and not re.search(r'^".*"$', text):
                # Si ce n'est pas déjà entouré de guillemets
                if "=" in text:
                    text = text.replace("=", '="', 1) + '"'
                else:
                    text = f'"{text}"'
            list_lex.append(text)
        return list_lex

    # exécution des commandes
    for line_cmd in commandes_lignes:
        if print_out:
            print(f"Exécution de la commande : {line_cmd}")
        commande = split_for_process_cmd(line_cmd)
        # ajout des arguments
        kwargs.update(dict(capture_output=True, text=True, check=True))
        try:
            result = subprocess.run(commande, **kwargs)
            if print_out & bool(result.stdout):
                print(f"Sortie de la commande : \n{result.stdout}")
        except subprocess.CalledProcessError as e:
            print(f"Erreur lors de l'exécution de la commande : {e}")
            print(f"Code de retour : {e.returncode}")
            print(f"Sortie d'erreur : {e.stderr}")
            print(f"output : {e.output}")
            break
        if print_out:
            print()

    # récupération de la sortie
    if return_out:
        try:
            return result.stdout
        except NameError:
            print("Aucune sortie à récupérer")
            return None


## WARNING ##
class ProjectNameExist(Warning):
    pass


VERSIONS = {}
part = ""
for version in versions:
    # récupération des parties
    if "Framework" in version:
        part = "Framework"
    elif "Engine" in version:
        part = "Engine"

    # récupération des versions
    if "Dart" in version:
        VERSIONS["dart"] = re.search(r"\d+\.\d+\.\d+", version).group()
    elif "Flutter" in version:
        VERSIONS["flutter"] = re.search(r"\d+\.\d+\.\d+", version).group()
    elif "DevTools" in version:
        VERSIONS["devTools"] = re.search(r"\d+\.\d+\.\d+", version).group()

    # récupération des historiques
    if part == "Framework":
        if " revision" in version:
            VERSIONS["framework_revision"] = re.search(r"ea121f8859", version).group()
        else:
            try:
                VERSIONS["framework_maj"] = re.search(r"\d+-\d+-\d+", version).group()
            except AttributeError:
                pass
    elif part == "Engine":
        if " revision" in version:
            VERSIONS["engine_revision"] = re.search(r"cf56914b32", version).group()
del part

In [ ]:
# Vérification de la disponibilité du nom du projet
projets = subprocess.run(
    # Récupération de la liste des noms des projets existants
    [
        "gh",
        "project",
        "list",
        "--owner",
        ORGANISATION,
        "--format",
        "json",
        "--jq",
        f".projects.[].title",
    ],
    capture_output=True,
    text=True,
    check=True,
)
if PROJET_NAME in re.split("\n", projets.stdout):
    raise ProjectNameExist(f"ATTENTION: le nom du projet '{PROJET_NAME}' existe déjà")

## 1.Création du projet Flutter

In [ ]:
# création du projet
process_cmd(
    [
        f'flutter create {PROJET_NAME} --platforms={",".join(PLATFORMS)} --description="{DESCRIPTION}" --org='
    ],
    kwargs=dict(cwd=CHEMIN_RACINE),
)

## 2.Initialisation Git

In [ ]:
process_cmd(
    ["git init"],
    kwargs=dict(cwd=CHEMIN_NEW_PROJET),
)

## 3.Editions des différents dossiers de Setup

### 1.Editions de l'arborescence de dossiers

#### 1.Dossier .gitgub

In [ ]:
# Créer le dossier .github
if not os.path.exists(GITHUB_FOLDER):
    os.makedirs(GITHUB_FOLDER)
# Créer le dossier templates
if not os.path.exists(TEMPLATES_FOLDER):
    os.makedirs(TEMPLATES_FOLDER)
# Créer le dossier workflows
if not os.path.exists(WORKFLOW_FOLDER):
    os.makedirs(WORKFLOW_FOLDER)

#### 2.Dossier assets

In [ ]:
# Création du dossier assets
if not os.path.exists(ASSETS_FOLDER):
    os.makedirs(ASSETS_FOLDER)
# Création du dossier images
if not os.path.exists(IMAGES_FOLDER):
    os.makedirs(IMAGES_FOLDER)

### 2.Editions du CI/CD

In [ ]:
# edition du CI:
with open(WORKFLOW_FOLDER + "CI.yml", "w", encoding="utf8") as f:
    f.write(
        f"""name: Flutter CI

# Declenche le workflow sur les pull_request
# pour les branches specifiees : main, develop et release/**
# lorsque les changements sont apportés dans le dossier lib assets ou pubspec.yaml
on:
  pull_request:
    branches:
      - main
      - develop
      - release**

jobs:
  # Dans un premier temps on fait l'analyse du code et les tests unitaires
  Analyse-et-tests:
    uses: Theobernardon/theo-flutter/.github/workflows/test-flutter.yml@v1

  # Ensuite on build l'application pour les plateformes specifiees
  Builds:
    needs: Analyse-et-tests
    uses: Theobernardon/theo-flutter/.github/workflows/builds.yml@v1
    with:
      # Liste des plateformes a builder
      platforms: '[{", ".join([f'"{platform}"' for platform in PLATFORMS])}]'
      mode: "debug" # release ou debug
"""
    )

In [ ]:
# edition du ReleaseMain:
with open(WORKFLOW_FOLDER + "ReleaseMain.yml", "w", encoding="utf8") as f:
    f.write(
        """name: Release (main)

on:
  workflow_dispatch:

permissions:
  contents: write

jobs:
  # Dans un premier temps on fait l'analyse du code et les tests unitaires
  Analyse-et-tests:
    uses: Theobernardon/theo-flutter/.github/workflows/test-flutter.yml@v1

  # Ensuite on build l'application
  Builds:
    needs: Analyse-et-tests
    uses: Theobernardon/theo-flutter/.github/workflows/builds.yml@v1
    with:
      # Liste des plateformes a builder
      platforms: '["linux", "windows"]'
      mode: "release" # release ou debug

  # Récupération de la version et création du Tag
  Tag:
    uses: Theobernardon/theo-flutter/.github/workflows/tag-version.yml@v1

  # Ensuite release le build de l'application
  Release:
    permissions:
      contents: write # for release
    needs: [Builds, Tag]
    uses: Theobernardon/theo-flutter/.github/workflows/release.yml@v1
    with:
      # Liste des plateformes a builder
      platforms: '["linux", "windows"]'
      version: ${{ needs.Tag.outputs.version }}
    secrets:
      token: ${{ secrets.GITHUB_TOKEN }}"""
    )

In [ ]:
# edition du MajDev:
with open(WORKFLOW_FOLDER + "MajDev.yml", "w", encoding="utf8") as f:
    f.write(
        """name: Mise à jour de develop

on:
  push:
    branches:
      - main

permissions:
  contents: write
  pull-requests: write

jobs:
  develop:
    uses: Theobernardon/theo-flutter/.github/workflows/majdevelop.yml@v1
"""
    )

### 3.Editions des fichées de màj de version

In [ ]:
# edition des modifier de version pubspec.yaml
with open(WORKFLOW_FOLDER + "MajPubspec.yml", "w", encoding="utf8") as f:
    f.write(
        """name: Bump version on release/hotfix

on:
  create:

permissions:
  contents: write

jobs:
  version:
    if: ${{ github.event.ref_type == 'branch' }}
    uses: Theobernardon/theo-flutter/.github/workflows/majpubspec.yml@v1
    with:
      branch_name: ${{ github.event.ref }}
"""
    )

In [ ]:
# edition du fichier changelog
with open(CHEMIN_NEW_PROJET + "CHANGELOG.md", "w", encoding="utf8") as f:
    f.write(
        f"""# Changelog

Toutes les modifications notables apportées à ce projet seront documentées dans ce dossier.

Ce projet utilise [Cider](https://pub.dev/packages/cider) pour automatiser la génération de changelogs basés sur [Conventional Commits](https://www.conventionalcommits.org/).

<!-- next-version-placeholder -->

## [Unreleased]
- Initial setup

"""
    )

### 5.Edition des issues templates a compléter

In [ ]:
# feature-request-template.yml
with open(
    TEMPLATES_FOLDER + "1-feature-request-template.yml", "w", encoding="utf8"
) as f:
    f.write(
        f"""name: "New feature (persona)"
description: "Décrire une fonctionnalité en fonction d’un besoin utilisateur"
title: "[Feature] - ..."
labels: ["feature"]
assignees: [Theobernardon]
type: "Feature"
projects: {ORGANISATION}/Projet_number
body:
  - type: dropdown
    id: persona
    attributes:
      label: 🧑 Persona
      description: Choisis un persona parmi ceux enregistrés
      options:
        - persona1 – description de persona1
        - persona2 – description de persona2
        - persona3 – description de persona3
        - persona4 – description de persona4
        - persona5 – description de persona5
      default: 0
  
  - type: textarea
    id: besoin
    attributes:
      label: 🎯 Veut... pour...
      description: "[Persona] veut... pour..."
      placeholder: "persona1 veut ... pour..."
  
  - type: textarea
    id: acceptance
    attributes:
      label: 📌 Critères d’acceptation
      description: Que doit faire la fonctionnalité au minimum pour être considérée comme terminée ?
      placeholder: |
        - Critères d’acceptation1
        - Critères d’acceptation2
        - ...
  
  - type: textarea
    id: test
    attributes:
      label: 🧪 Scénarios de test utilisateur (optionnel)
      placeholder: |
        - test1
        - test2
        - ...
"""
    )

In [ ]:
# sub-feature-request-template.yml
with open(
    TEMPLATES_FOLDER + "2-sub-feature-request-template.yml", "w", encoding="utf8"
) as f:
    f.write(
        f"""name: "New sub feature (persona)"
description: "Décrire une fonctionnalité en fonction d’un besoin utilisateur"
title: "[SubFeature] - "
labels: ["subfeature"]
assignees: [Theobernardon]
type: "Feature"
projects: {ORGANISATION}/Projet_number
body:
  - type: dropdown
    id: persona
    attributes:
      label: 🧑 Persona
      description: Choisis un persona parmi ceux enregistrés
      options:
        - persona1 – description de persona1
        - persona2 – description de persona2
        - persona3 – description de persona3
        - persona4 – description de persona4
        - persona5 – description de persona5
      default: 0
  
  - type: textarea
    id: besoin
    attributes:
      label: 🎯 Veut... pour...
      description: "[Persona] veut... pour..."
      placeholder: "persona1 veut ... pour..."
  
  - type: textarea
    id: acceptance
    attributes:
      label: 📌 Critères d’acceptation
      description: Que doit faire la fonctionnalité au minimum pour être considérée comme terminée ?
      placeholder: |
        - Critères d’acceptation1
        - Critères d’acceptation2
        - ...
  
  - type: textarea
    id: test
    attributes:
      label: 🧪 Scénarios de test utilisateur (optionnel)
      placeholder: |
        - test1
        - test2
        - ...
"""
    )

In [ ]:
# bug-report.yml
with open(TEMPLATES_FOLDER + "3-bug-report.yml", "w", encoding="utf8") as f:
    f.write(
        f"""name: "Bug report"
description: "Créer un rapport pour nous aider à nous améliorer"
title: "[BUG] - "
labels: ["bug"]
assignees: [Theobernardon]
type: "Bug"
projects: {ORGANISATION}/Projet_number
body:
  - type: textarea
    id: description
    attributes:
      label: 🐛 Décrire le bug
      description: "Décrire le bug de manière claire et concise"
      placeholder: "Décrire le bug ici..."
  
  - type: textarea
    id: reproduction
    attributes:
      label: 🔁 Étapes pour reproduire le bug
      description: "Décrire les étapes pour reproduire le bug"
      placeholder: |
        1. Allez à '...'
        2. Cliquez sur '....'
        3. Faites défiler l'écran jusqu'à '....'
        4. Voir l'erreur
  
  - type: textarea
    id: attente
    attributes:
      label: 🧪 Ce à quoi je m'attendais
      description: "Décrire ce que vous attendiez"
      placeholder: "Une description claire et concise de ce que vous attendez ici..."

  - type: markdown
    id: photos
    attributes:
      value: |
        ### 📸 Photos / liens / ...
        Ajouter des **photos** pour illustrer le bug en draguant & drop ici ou  
        en cliquant sur le bouton ci-dessous. N'hésitez pas non plus à lier du  
        code ou ce qui vous semble nécessaire pour la bonne compréhension du bug"
      
"""
    )

In [ ]:
# fix-issue.yml
with open(TEMPLATES_FOLDER + "4-fix-issue.yml", "w", encoding="utf8") as f:
    f.write(
        f"""name: "Fix issue"
description: "Correction de bugs"
title: "[FIX] - "
labels: ["fix"]
assignees: [Theobernardon]
type: "Bug"
projects: {ORGANISATION}/Projet_number
body:
  - type: textarea
    id: description
    attributes:
      label: 🐛 Décrire le bug
      description: "Décrire le bug et le liée (si possible)"
      placeholder: "Décrire le bug ici..."
  
  - type: textarea
    id: pistes
    attributes:
      label: 🔍 Piste de résolution
      description: "Quelques idées de résolutions à chaud"
      placeholder: "Décrire la piste de résolution ici..."
  
  - type: textarea
    id: acceptance
    attributes:
      label: 📌 Critères d’acceptation
      description: Que doit faire la fonctionnalité au minimum pour être considérée comme terminée ?
      placeholder: |
        - Critères d’acceptation1
        - Critères d’acceptation2
        - ...
  
  - type: textarea
    id: test
    attributes:
      label: 🧪 Scénarios de test utilisateur (optionnel)
      placeholder: |
        - test1
        - test2
        - ...
"""
    )

In [ ]:
# hotfix-issue.yml
with open(TEMPLATES_FOLDER + "5-hotfix-issue.yml", "w", encoding="utf8") as f:
    f.write(
        f"""name: "Hotfix issue"
description: "Correction de bugs"
title: "[HOTFIX] - "
labels: ["hotfix"]
assignees: [Theobernardon]
type: "Bug"
projects: {ORGANISATION}/Projet_number
body:
  - type: textarea
    id: description
    attributes:
      label: 🐛 Décrire le bug
      description: "Décrire le bug et le liée (si possible)"
      placeholder: "Décrire le bug ici..."
  
  - type: textarea
    id: pistes
    attributes:
      label: 🔍 Piste de résolution
      description: "Quelques idées de résolutions à chaud"
      placeholder: "Décrire la piste de résolution ici..."
  
  - type: textarea
    id: acceptance
    attributes:
      label: 📌 Critères d’acceptation
      description: Que doit faire la fonctionnalité au minimum pour être considérée comme terminée ?
      placeholder: |
        - Critères d’acceptation1
        - Critères d’acceptation2
        - ...
  
  - type: textarea
    id: test
    attributes:
      label: 🧪 Scénarios de test utilisateur (optionnel)
      placeholder: |
        - test1
        - test2
        - ...
"""
    )

In [ ]:
# doc-issue.yml
with open(TEMPLATES_FOLDER + "6-doc-issue.yml", "w", encoding="utf8") as f:
    f.write(
        f"""name: "Documentation issue"
description: "Créer un rapport pour nous aider à nous améliorer"
title: "[DOC] - "
labels: ["doc"]
assignees: Theobernardon
type: "Task"
projects: {ORGANISATION}/Projet_number
body:
  - type: textarea
    id: description
    attributes:
      label: 📝 Décrire le besoin
      description: "Décrire le besoin de documentation"
      placeholder: "Décrire le besoin ici..."
"""
    )

### 6.Import le script d'Auto édit Projet_number

In [ ]:
# On doit juste copier coller le contenu de
# resources\issue_template\Projet_number_edit.py dans .github\ISSUE_TEMPLATE
shutil.copy2(
    os.path.abspath("resources\\issue_template\\Projet_number_edit.py"),
    TEMPLATES_FOLDER,
)

Une fois le **Github Project** crée activez le script **Projet_number_edit.py** pour changer le nombre du projet dans les différents fichiers issus Template, Et ainsi relier automatiquement les nouvelles issues au projet en cours

## 4.innosetup si windose in platformes

In [ ]:
if "windows" in PLATFORMS:
    MyAppId = generate_guid()
    MyAppName = PROJET_NAME.replace("_", " ").capitalize()
    MyAppPublisher = ORGANISATION
    MyAppURL = ""
    MyAppExeName = f"{PROJET_NAME}.exe"
    MyAppAssocName = ""
    MyAppAssocExt = ""
    MyAppAssocKey = MyAppAssocName.replace(" ", "") + MyAppAssocExt

    # édition du fichier innosetup
    with open(INNOSETUP_FILE, "w", encoding="utf8") as f:
        f.write(
            f'''#ifndef MyAppSetupName
  #define MyAppSetupName "SetupFile"
#endif
#ifndef MyAppVersion
  #error "La variable MyAppVersion n'est pas définie !"
#endif


[Setup]
AppId={"{{"}{MyAppId}{"}}"}
AppName={MyAppName}
AppVersion={"{#MyAppVersion}"}
AppPublisher={MyAppPublisher}
AppPublisherURL={MyAppURL}
AppSupportURL={MyAppURL}
AppUpdatesURL={MyAppURL}
DefaultDirName={"{autopf}"}\\{MyAppName}
ArchitecturesAllowed=x64compatible
ArchitecturesInstallIn64BitMode=x64compatible
ChangesAssociations=yes
DisableProgramGroupPage=yes
PrivilegesRequired=lowest
OutputDir=installer/output
OutputBaseFilename={"{#MyAppSetupName}"}
UninstallDisplayIcon=assets\\images\\uninstall_icon.ico
SetupIconFile=assets\\images\\setup_icon.ico
SolidCompression=yes
WizardStyle=modern

[Languages]
Name: "english"; MessagesFile: "compiler:Default.isl"
Name: "french"; MessagesFile: "compiler:Languages\\French.isl"

[Tasks]
Name: "desktopicon"; Description: "{"{cm:CreateDesktopIcon}"}"; GroupDescription: "{"{cm:AdditionalIcons}"}"; Flags: unchecked

[Files]
Source: "build\\windows\\x64\\runner\\Release\\*"; DestDir: "{"{app}"}"; Flags: recursesubdirs createallsubdirs
; NOTE: Don't use "Flags: ignoreversion" on any shared system files

[Registry]
Root: HKCU; Subkey: "Software\\Classes\\{MyAppAssocExt}\\OpenWithProgids"; ValueType: string; ValueName: "{MyAppAssocKey}"; ValueData: ""; Flags: uninsdeletevalue
Root: HKCU; Subkey: "Software\\Classes\\{MyAppAssocKey}"; ValueType: string; ValueName: ""; ValueData: "{MyAppAssocName}"; Flags: uninsdeletekey
Root: HKCU; Subkey: "Software\\Classes\\{MyAppAssocKey}\\DefaultIcon"; ValueType: string; ValueName: ""; ValueData: "{"{app}"}\\{MyAppExeName},0"
Root: HKCU; Subkey: "Software\\Classes\\{MyAppAssocKey}\\shell\\open\\command"; ValueType: string; ValueName: ""; ValueData: """{"{app}"}\\{MyAppExeName}"" ""%1"""

[Icons]
Name: "{"{autoprograms}"}\\{MyAppName}"; Filename: "{"{app}"}\\{MyAppExeName}"
Name: "{"{autodesktop}"}\\{MyAppName}"; Filename: "{"{app}"}\\{MyAppExeName}"; Tasks: desktopicon

[Run]
Filename: "{"{app}"}\\{MyAppExeName}"; Description: "{"{cm:LaunchProgram,"}{MyAppName.replace("&", "&&")}{"}"}"; Flags: nowait postinstall skipifsilent
'''
        )

## 5.Gestion des icônes

*Je profite de cette partie où je dois te toucher au pubspec.yaml pour renseigner la version actuelle de flutter*

Les icônes doivent être mis dans le dossier asset\image au format PNG en respectant les règles de nommages:  
  
- android:
    - android_icon.png
    - android_adapt_back_icon.png
    - android_adapt_fore_icon.png
    - android_adapt_mono_icon.png
- ios:
    - ios_icon.png
    - ios_dark_transparent_icon.png
    - ios_tinted_grayscale_icon.png
- autres:
    - app_icon.png
  
Puis, exécuter les 2 commandes suivantes pour générer les icons  
```bash
flutter pub get
flutter pub run flutter_launcher_icons:main
```

In [ ]:
### Gestion des icônes avec le package flutter_launcher_icons ###
# Ouverture du fichier pubspec et extraction du contenu yaml
with open(PUBSPEC_FILE, "r", encoding="utf8") as f:
    data = yaml.safe_load(f)

# Définition de la version de flutter_launcher_icons
data.setdefault("dev_dependencies", {})["flutter_launcher_icons"] = "^0.13.1"

# Paramétrage du package flutter_launcher_icons en fonction des plateformes
param_icons = {"image_path": f"{IMAGES_RELATIF_FOLDER}app_icon.png"}
for platform in PLATFORMS:
    match platform:
        # https://pub.dev/documentation/flutter_launcher_icons/latest/
        case "android":
            param_icons["flutter_launcher_icons"] = {
                "android": True,
                "image_path_android": f"{IMAGES_RELATIF_FOLDER}android_icon.png",
                "adaptive_icon_background": f"{IMAGES_RELATIF_FOLDER}android_adapt_back_icon.png",
                "adaptive_icon_foreground": f"{IMAGES_RELATIF_FOLDER}android_adapt_fore_icon.png",
                "adaptive_icon_foreground_inset": 16,
                "adaptive_icon_monochrome": f"{IMAGES_RELATIF_FOLDER}android_adapt_mono_icon.png",
            }
        case "ios":
            # les icônes iOS doivent remplir toute l’image et ne pas contenir de bordures transparentes.
            param_icons["flutter_launcher_icons"] = {
                "ios": True,
                "image_path_ios": f"{IMAGES_RELATIF_FOLDER}ios_icon.png",
                "remove_alpha_ios": True,
                "background_color_ios": "#ffffff",
                "image_path_ios_dark_transparent": f"{IMAGES_RELATIF_FOLDER}ios_dark_transparent_icon.png",
                "image_path_ios_tinted_grayscale": f"{IMAGES_RELATIF_FOLDER}ios_tinted_grayscale_icon.png",
                "desaturate_tinted_to_grayscale_ios": True,
            }
        case "windows":
            param_icons["windows"] = {
                "generate": True,
                # "image_path": f"{IMAGES_RELATIF_FOLDER}windows_icon.png",
                "icon_size": 256,
            }
        case "macos":
            param_icons["macos"] = {
                "generate": True,
                # "image_path": f"{IMAGES_RELATIF_FOLDER}macos_icon.png",
            }
        case "web":
            param_icons["web"] = {
                "generate": True,
                # "image_path": f"{IMAGES_RELATIF_FOLDER}web_icon.png",
            }
        case "linux":
            pass
        case _:
            raise ValueError(f"Plateforme inconnue : {platform}")
data["flutter_launcher_icons"] = param_icons
# Ajout de la version de flutter dans l'environnement
data["environment"]["flutter"] = VERSIONS["flutter"]
# Edition des paramètres dans le pubspec.yml
with open(PUBSPEC_FILE, "w", encoding="utf8") as f:
    yaml.dump(data, f, sort_keys=False, allow_unicode=True)

In [ ]:
# Gestion spéciale supplémentaire pour les icons innosetup install/unisntall
if "windows" in PLATFORMS:
    # On doit juste copier coller le contenu de ressources\icons dans asset\images
    shutil.copytree(
        os.path.abspath("resources/icons/"), IMAGES_FOLDER, dirs_exist_ok=True
    )

## 6.Github repo

### 1.création du repo

In [ ]:
# Création du Repo github
process_cmd(
    [
        f"gh repo create {ORGANISATION}/{PROJET_NAME} --{DISPONIBILITE} --source=.",
        "git add .",
        'git commit -m "first commit"',
        "git branch -M main",
        f"git remote set-url origin {GITHUB_PROJECT}.git",
        "git push -u origin main",
        f"gh repo edit {GITHUB_PROJECT} --delete-branch-on-merge=true",
    ],
    kwargs=dict(cwd=CHEMIN_NEW_PROJET),
)

### 2.édition des tags

In [ ]:
# Récupération de la liste des labels automatiquement créés par GitHub
list_label = process_cmd(
    [
        f"gh label list --repo {GITHUB_PROJECT}",
    ],
    kwargs=dict(cwd=CHEMIN_NEW_PROJET),
    return_out=True,
    print_out=False,
)
# On retravaille un peu pour obtenir une réelle liste avec les labels et non pas une chaîne de caractères
list_label = list_label.split("\n")
list_label = [re.split(r"\t", label)[0] for label in list_label]
list_label.remove("")
list_label
# On supprime tous les labels préexistants
for label in list_label:
    subprocess.run(
        [
            "gh",
            "label",
            "delete",
            label,
            "--repo",
            GITHUB_PROJECT,
            "--yes",
        ],
    )
# Enfin il nous reste plus qu'à ajouter nos labels
process_cmd(
    [
        f'gh label create bug --description "Rapport de bug à transformer en fix issue" --color B60205 --repo {GITHUB_PROJECT}',
        f'gh label create feature --description "Développement de nouvelle fonctionnalité" --color 8E01C0 --repo {GITHUB_PROJECT}',
        f'gh label create subfeature --description "Sous fonctionnalités d\'une feature" --color 8E01C0 --repo {GITHUB_PROJECT}',
        f'gh label create hotfix --description "Résolution d\'un bug prioritaire sur la branche release ou develop" --color D93F0B --repo {GITHUB_PROJECT}',
        f"gh label create fix --description \"Résolution d'un bug ou d'un dysfonctionnement quelconque\" --color F1FA9D --repo {GITHUB_PROJECT}",
        f'gh label create doc --description "Afférent à la documentation" --color eeeeee --repo {GITHUB_PROJECT}',
    ],
    kwargs=dict(cwd=CHEMIN_NEW_PROJET),
)

In [ ]:
# Création de la branche develop
process_cmd(
    [
        "git checkout -b develop",
        "git push -u origin develop",
    ],
    kwargs=dict(cwd=CHEMIN_NEW_PROJET),
)

## 7.Github projet

Pour l'instant les projets ne peuvent pas être gérés en ligne de commande avec l'interface github CLI. J'utilise donc un projet Template comme automatisation partielle de la création d'un nouveau projet.  
*A noter que je suis obligé de passer par une organisation car seuls les organisations ont la possibilité de créer des templates de projet*

Une fois le projet créé, récupérer le ***n° du projet*** (#xx) et Exécuter le script Python ```Projet_number_edit.py``` dans votre dossier issue template. Ceci va permettre de faire le lien automatiquement entre votre projet github et les différentes issues créées par les templates.